In [4]:
import pandas as pd
import numpy as np

from PIL import Image
import cv2

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import torch
print(torch.__version__)
print(torch.cuda.is_available())

import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch.nn as nn
from torch import optim

from torchsummary import summary

2.0.1+cu117
True


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() == True else 'cpu')
device
# device = 'cpu'

device(type='cuda')

In [6]:
# img_path = r'C:\Users\user\ML\endoscopy\kvasir-capsule'
# train_df = pd.read_csv(img_path+'/train.csv')
# validate_df = pd.read_csv(img_path+'/validation.csv')
# test_df = pd.read_csv(img_path+'/test.csv')

# img_path = r'C:\Users\user\ML\endoscopy\kvasir-capsule-augments'
# train_df = pd.read_csv(img_path+'/train_augment_polyp.csv')
# validate_df = pd.read_csv(img_path+'/validation_augment_polyp.csv')
# test_df = pd.read_csv(img_path+'/test_augment_polyp.csv')

# img_path = r'C:\Users\user\ML\endoscopy\kvasir-capsule-augments'
# train_df = pd.read_csv(img_path+'/train_augment_polyp_40.csv')
# validate_df = pd.read_csv('C:/Users/user/ML/endoscopy/kvasir-capsule/validation.csv')
# test_df = pd.read_csv('C:/Users/user/ML/endoscopy/kvasir-capsule/test.csv')

img_path = r'C:\Users\user\ML\endoscopy\train_validation_test_sets'
train_df = pd.read_excel(img_path+'/kvasir-polyps-combined_set1.xlsx', sheet_name='train')
#validate_df = pd.read_excel(img_path+'/kvasir-polyps-combined_set1.xlsx', sheet_name='validate')
validate_df = pd.read_csv(img_path+'/kvasir-polyps-combined_val_balanced200.csv')
test_df = pd.read_excel(img_path+'/kvasir-polyps-combined_set1.xlsx', sheet_name='test')

classes = tuple(set(train_df['label']))
print(classes)
idx_to_class = {i:j for i, j in enumerate(classes)}
class_to_idx = {value:key for key,value in idx_to_class.items()}
class_to_idx

('Erosion', 'Ampulla of Vater', 'adenomatous', 'Lymphangiectasia', 'hyperplastic', 'Blood - hematin', 'Erythema', 'Pylorus', 'Normal clean mucosa', 'missing', 'Angiectasia', 'Reduced Mucosal View', 'Ileocecal valve', 'Ulcer', 'Foreign Body', 'Blood - fresh')


{'Erosion': 0,
 'Ampulla of Vater': 1,
 'adenomatous': 2,
 'Lymphangiectasia': 3,
 'hyperplastic': 4,
 'Blood - hematin': 5,
 'Erythema': 6,
 'Pylorus': 7,
 'Normal clean mucosa': 8,
 'missing': 9,
 'Angiectasia': 10,
 'Reduced Mucosal View': 11,
 'Ileocecal valve': 12,
 'Ulcer': 13,
 'Foreign Body': 14,
 'Blood - fresh': 15}

In [7]:
train_df['label'].value_counts()

Normal clean mucosa     24680
adenomatous             16798
Ileocecal valve         15103
hyperplastic            15093
Reduced Mucosal View    10448
Pylorus                  5535
Angiectasia              3121
Ulcer                    3050
Foreign Body             2786
Lymphangiectasia         2139
Erosion                  1822
missing                  1746
Blood - fresh            1605
Erythema                  571
Blood - hematin            45
Ampulla of Vater           38
Name: label, dtype: int64

In [8]:
# hyperparameters
input_size = (229*229)
num_classes = len(classes)
learning_rate = 0.001
batch_size = 64
num_epochs = 8

In [9]:
vgg16 = models.vgg16(weights='VGG16_Weights.DEFAULT')
num_ftrs = vgg16.fc.in_features
vgg16.fc = nn.Linear(num_ftrs, len(classes))
summary(vgg16,(3,229,229))

AttributeError: 'VGG' object has no attribute 'fc'

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
     ])

# mean and std dev of imagenet are - Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))

In [ ]:
train_image_paths = []
train_image_labels = []
train_image_classes = []

for i in range(len(train_df)):
    train_image_paths.append(train_df.loc[i,'filename'])
    train_image_labels.append(train_df.loc[i,'label'])
    train_image_classes.append(class_to_idx[train_df.loc[i,'label']])

validate_image_paths = []
validate_image_labels = []
validate_image_classes = []

for i in range(len(validate_df)):
    validate_image_paths.append(validate_df.loc[i,'filename'])
    validate_image_labels.append(validate_df.loc[i,'label'])
    validate_image_classes.append(class_to_idx[validate_df.loc[i,'label']])

test_image_paths = []
test_image_labels = []
test_image_classes = []

for i in range(len(test_df)):
    test_image_paths.append(test_df.loc[i,'filename'])
    test_image_labels.append(test_df.loc[i,'label'])
    test_image_classes.append(class_to_idx[test_df.loc[i,'label']])
    
# for i in range(len(train_image_paths)):
#     print(train_image_paths[i] + ' - ' + train_image_labels[i] + ' - ' + str(train_image_classes[i]))

In [ ]:
class EndoscopyDataset(Dataset):
    def __init__(self, image_paths, image_classes, transform=False):
        self.image_paths = image_paths
        self.image_classes = image_classes
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        
        image_path = self.image_paths[idx]
        #print(image_path)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        dim = (229, 229)
        image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        
        label = self.image_classes[idx]
        label = torch.tensor(label)

        if self.transform is not None:
            image = self.transform(image)
        
        return image, label

# Create Dataset
train_dataset = EndoscopyDataset(train_image_paths,train_image_classes,transform)
validate_dataset = EndoscopyDataset(validate_image_paths,validate_image_classes,transform) 
test_dataset = EndoscopyDataset(test_image_paths,test_image_classes,transform)

In [ ]:
df_results = pd.DataFrame(columns=['Epoch', 'Loss', 'accuracy training', 'accuracy validation'])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(validate_dataset, batch_size=batch_size, shuffle=True)
 
class EndoscopyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.3)
 
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3,3), stride=1, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))
 
        self.flat = nn.Flatten()
 
        self.fc3 = nn.Linear(831744, 512)
        self.act3 = nn.ReLU()
        self.drop3 = nn.Dropout(0.5)
 
        self.fc4 = nn.Linear(512, len(classes))
 
    def forward(self, x):
        # input 3x229x229, output 32x229x229
        x = self.act1(self.conv1(x))
        x = self.drop1(x)
        # input 32x229x229, output 63x229x229
        x = self.act2(self.conv2(x))
        # input 64x229x229, output 64x114x114
        x = self.pool2(x)
        # input 64x114x114, output 831744
        x = self.flat(x)
        # input 831744, output 512
        x = self.act3(self.fc3(x))
        x = self.drop3(x)
        # input 512, output 10
        x = self.fc4(x)
        return x
 

model = vgg16 #EndoscopyModel()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        #print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples)}')
        
    return float(num_correct) / float(num_samples)

# train network
for epoch in range(num_epochs):
    losses = []
    
    for batch_idx, (data, targets) in enumerate(trainloader):
        
        # get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        scores = model(data)
        loss = criterion(scores, targets)
        
        losses.append(loss.item())
    
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
    train_accuracy = check_accuracy(trainloader, model)
    validation_accuracy = check_accuracy(testloader, model)
    print(f'{epoch}, {sum(losses)/len(losses)}, {train_accuracy}, {validation_accuracy}')
    
    df_results = pd.concat([df_results,
                           pd.DataFrame({'Epoch':epoch, 'Loss': losses, 'accuracy training':train_accuracy, 
                                         'accuracy validation':validation_accuracy})], ignore_index=True)

        
model.train()
    
print('checking accuracy on the training set', check_accuracy(trainloader, model))

print('checking accuracy on the test set', check_accuracy(testloader, model))

torch.save(model.state_dict(), "endoscopymodel_testset1.pth")

In [ ]:
# for epoch in range(n_epochs):
#     for image, label in trainloader:
#         # forward, backward, and then weight update
#         y_pred = model(image)
#         loss = loss_fn(y_pred, label)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
 
#     acc = 0
#     count = 0
#     for image, label in testloader:
#         y_pred = model(image)
#         acc += (torch.argmax(y_pred, 1) == labels).float().sum()
#         count += len(labels)
#     acc /= count
#     print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))

In [ ]:
print('checking accuracy on the training set', check_accuracy(trainloader, model))

print('checking accuracy on the test set', check_accuracy(testloader, model))

torch.save(model.state_dict(), "endoscopymodel_testset_64_8epochs_valbal_imagenetnorm.pth")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(df_results['Epoch'], df_results['accuracy training'], color='green', marker='o', linestyle='dashed', linewidth=1, markersize=4, label='training')
plt.plot(df_results['Epoch'], df_results['accuracy validation'], color='blue', marker='o', linestyle='dashed', linewidth=1, markersize=4, label='validation')
plt.legend()
plt.show()